# Seam carving

### Tengo
- Una matriz w * h (Una imagen con pixeles)
- Cada celda tiene un valor v(n,m) (Cada pixel tiene una energia asociada)

### Quiero
- Encontrar la veta (Horizontal o vertical) de menor "energia"
Es decir, el camino de 0 a w, o de 0 a h cuya suma de valores sea la minima. De una celda me puedo desplazar a sus 3 próximos alrededores dependiendo la direccion (horizontal o vertical).

### Recurrencia
- Desde una celda puedo acceder solo a otras 3 (2 para los casos de los bordes)
- A una celda se puede ser accedido solo desde otras 3 (2 para los casos de los bordes)

Veamos un caso horizontal:
- En la primera columna, el valor acumulado sería solo el valor de la celda dada.
- Para una columna generica, el valor acumulado sería el valor de la celda más lo acumulado hasta llegar a ella. Recordemos que solo se puede llegar hacia una celda desde 3 (o 2 para los bordes) celdas de la columna anterior

siendo i el indice de fila y j el indice de columna
OPT(i, j) = vi,  si j = 0
OPT(i, j) = vi + Min { OPT(i - 1 , j - 1), OPT(i, j - 1), OPT(i + 1 , j - 1) }, si j > 0

Y el resultado será el min entre los caminos minimos de haber llegado hasta los píxeles de la última columna, es decir min { OPT(i, w) }.


In [10]:
IMG = [
    [0, 2, 2, 0],
    [2, 4, 4, 0],     
    [0, 4, 4, 0],  # => 0, 1, 1, 0 debe ser la veta horizontal
    [1, 1, 1, 1],
]
def seam_carving_horizontal(img):
    h = len(img)
    w = len(img[0])
    opt = [[0] * w for _ in range(h)]
    anterior_elegido = [[0] * w for _ in range(h)]
    selecciones = []

    for i in range(h):
        opt[i][0] = img[i][0]
    
    for j in range(1, w):
        for i in range(0, h):
            # valor actual
            vij = img[i][j] 
            
            # valores acumulados de las 3 posibles celdas anteriores
            acumulado_desde_anterior_arriba = float('inf')
            acumulado_desde_anterior_abajo = float('inf')
            if i > 0:
                acumulado_desde_anterior_arriba = opt[i - 1][j - 1]
            acumulado_desde_anterior = opt[i][j - 1]
            if i < h - 1:
                acumulado_desde_anterior_abajo = opt[i + 1][j - 1]
                
            # La celda de las 3 anteriores que menos acumulo es:
            movimiento_de_fila_anterior = 0 # -1 una fila arriba, 0 misma fila, 1 una fila abajo
            min_acumulado_entre_anteriores = min(
                acumulado_desde_anterior_arriba,
                acumulado_desde_anterior,
                acumulado_desde_anterior_abajo
            )
            if min_acumulado_entre_anteriores == acumulado_desde_anterior_arriba:
                movimiento_de_fila_anterior = -1
            elif min_acumulado_entre_anteriores == acumulado_desde_anterior_abajo:
                movimiento_de_fila_anterior = 1
                
            # Asignaciones
            opt[i][j] = vij + min_acumulado_entre_anteriores
            anterior_elegido[i][j] = movimiento_de_fila_anterior
    
    # Obtener los valores de la última columna junto con sus índices
    v_acumulados_ultima_columna = [(fila[-1], idx) for idx, fila in enumerate(opt)]
    # Encontrar el valor mínimo y la fila correspondiente
    menor_v_acumulado_de_celdas_de_ultima_columna, fila_del_menor_valor = min(v_acumulados_ultima_columna)
    
    # Recontruccion camino
    j = w - 1
    i = fila_del_menor_valor
    while j >= 0:
        selecciones.append((i,j))
        i += anterior_elegido[i][j]
        j -= 1
    
    return menor_v_acumulado_de_celdas_de_ultima_columna, selecciones
            
seam_carving_horizontal(IMG)

(2, [(2, 3), (3, 2), (3, 1), (2, 0)])

### Complejidad
#### Temporal: 
1) Para todos los pixeles de la columna 1 seteo el caso base -> O(h)
2) Para cada columna recorro cada fila -> O(h * w)
3) Recorro los valores acumulados de los pixeles de la ultima columna -> O(h)
=> O(h * w)
#### Espacial: Espacial guardo los optimos de cada celda => O(h * w)